In [1]:
import pandas as pd
import os

In [2]:
broken_mains = pd.read_csv('../data/raw/Watermain_Attributes_Soil_Ph.csv')

broken_mains = broken_mains[['FACILITYID', 'LOCATION', 'INSTALLDAT', 'SUBTYPE',
       'MATERIAL', 'LENGTH', 'DIAMETER', 'STATUS', 'PressureSy']].drop_duplicates()
broken_mains['INSTALLDAT'] = pd.to_datetime(broken_mains['INSTALLDAT'], format='mixed')
broken_mains.info()

<class 'pandas.core.frame.DataFrame'>
Index: 855 entries, 0 to 875
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   FACILITYID  855 non-null    object        
 1   LOCATION    855 non-null    object        
 2   INSTALLDAT  847 non-null    datetime64[ns]
 3   SUBTYPE     855 non-null    int64         
 4   MATERIAL    855 non-null    object        
 5   LENGTH      855 non-null    float64       
 6   DIAMETER    855 non-null    float64       
 7   STATUS      855 non-null    object        
 8   PressureSy  855 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 66.8+ KB


In [3]:
broken_mains['break'] = 'yes'

In [4]:
broken_mains[['FACILITYID', 'LOCATION', 'INSTALLDAT', 'SUBTYPE',
       'MATERIAL', 'LENGTH', 'DIAMETER', 'STATUS', 'PressureSy']].head()

,FACILITYID,LOCATION,INSTALLDAT,SUBTYPE,MATERIAL,LENGTH,DIAMETER,STATUS,PressureSy
0,22-23992,Arborland Mall,1961-09-11,1,CI,0.0,8.0,IS,SHE
1,00-00079,Marlborough,1962-06-30,2,CI,453.0,20.0,IS,SHE
2,00-00404,Platt Rd,1978-07-24,2,DI,0.0,16.0,IS,SHE
3,00-00428,Platt Rd,1978-07-24,2,DI,0.0,16.0,IS,SHE
4,00-00429,Platt Rd,1960-01-01,1,CI,160.0,8.0,IS,SHE


In [5]:
all_mains = pd.read_csv('../data/raw/All_Watermains_Attributes.csv', encoding='latin-1').drop_duplicates()
all_mains['INSTALLDAT'] = pd.to_datetime(all_mains['INSTALLDAT'], format='mixed')
all_mains['break'] = 'no'
all_mains.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31332 entries, 0 to 32652
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ENABLED     31332 non-null  int64         
 1   FACILITYID  31047 non-null  object        
 2   LOCATION    17788 non-null  object        
 3   INSTALLDAT  28905 non-null  datetime64[ns]
 4   SUBTYPE     31332 non-null  int64         
 5   MATERIAL    31296 non-null  object        
 6   LENGTH      31332 non-null  float64       
 7   DIAMETER    31332 non-null  float64       
 8   STATUS      31332 non-null  object        
 9   PressureSy  30487 non-null  object        
 10  break       31332 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 2.9+ MB


In [6]:
all_mains.head()

,ENABLED,FACILITYID,LOCATION,INSTALLDAT,SUBTYPE,MATERIAL,LENGTH,DIAMETER,STATUS,PressureSy,break
0,1,22-00008,Packard St,1963-05-16,3,CI,0.0,6.0,IS,SHE,no
1,1,22-00016,Packard St,1950-01-01,3,CI,6.0,6.0,IS,SHE,no
2,1,22-00017,Packard St,1950-01-01,3,CI,17.0,6.0,IS,SHE,no
3,1,22-00048,Packard St,1930-01-01,3,CI,18.0,6.0,IS,SHE,no
4,1,22-00115,Eastover Pl,1940-01-01,3,CI,25.0,6.0,IS,SHE,no


In [19]:
common_columns = set(all_mains.columns).intersection(set(broken_mains.columns))
common_columns

{'DIAMETER',
 'FACILITYID',
 'INSTALLDAT',
 'LENGTH',
 'LOCATION',
 'MATERIAL',
 'PressureSy',
 'STATUS',
 'SUBTYPE',
 'break'}

In [8]:
merging_columns = common_columns - \
{
    # 'DIAMETER',
 # 'FACILITYID',
 # 'INSTALLDAT',
 # 'LENGTH',
 'LOCATION',
 # 'MATERIAL',
 # 'PressureSy',
 # 'STATUS',
 # 'SUBTYPE'
 }

In [9]:
all_mains.merge(broken_mains, on=list(merging_columns), how='inner').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ENABLED     0 non-null      int64         
 1   FACILITYID  0 non-null      object        
 2   LOCATION_x  0 non-null      object        
 3   INSTALLDAT  0 non-null      datetime64[ns]
 4   SUBTYPE     0 non-null      int64         
 5   MATERIAL    0 non-null      object        
 6   LENGTH      0 non-null      float64       
 7   DIAMETER    0 non-null      float64       
 8   STATUS      0 non-null      object        
 9   PressureSy  0 non-null      object        
 10  break       0 non-null      object        
 11  LOCATION_y  0 non-null      object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 132.0+ bytes


In [ ]:
broken_mains

In [14]:
all_mains['LOCATION'].info(), all_mains.shape

<class 'pandas.core.series.Series'>
Index: 31332 entries, 0 to 32652
Series name: LOCATION
Non-Null Count  Dtype 
--------------  ----- 
17788 non-null  object
dtypes: object(1)
memory usage: 489.6+ KB


(None, (31332, 11))

In [26]:
df = pd.concat([broken_mains[list(common_columns)], all_mains[list(common_columns)]])

In [24]:
df.to_csv("../data/all_watermains.csv

,FACILITYID,MATERIAL,SUBTYPE,STATUS,break,INSTALLDAT,LENGTH,LOCATION,DIAMETER,PressureSy
0,22-00008,CI,3,IS,no,1963-05-16,0.0,Packard St,6.0,SHE
1,22-00016,CI,3,IS,no,1950-01-01,6.0,Packard St,6.0,SHE
2,22-00017,CI,3,IS,no,1950-01-01,17.0,Packard St,6.0,SHE
3,22-00048,CI,3,IS,no,1930-01-01,18.0,Packard St,6.0,SHE
4,22-00115,CI,3,IS,no,1940-01-01,25.0,Eastover Pl,6.0,SHE
...,...,...,...,...,...,...,...,...,...,...
32648,00-77070,DI,1,IS,no,NaT,0.0,NaN,8.0,NaN
32649,00-76868,DI,1,IS,no,2023-05-01,237.0,S State St,12.0,GRA
32650,00-76912,DI,1,IS,no,2023-05-01,36.0,S State St & Washington St,12.0,GRA
32651,00-77041,DI,3,IS,no,2023-07-18,13.0,Olivia Ave,6.0,GRA


In [ ]:
os.